In [ ]:
import sys
sys.path.append("/scratch/group/csce435-f24/python-3.10.4/lib/python3.10/site-packages")
sys.path.append("/scratch/group/csce435-f24/thicket")
from glob import glob

import matplotlib.pyplot as plt
import pandas as pd

import thicket as th

pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

In [ ]:
#1_trial is a name of a folder containing the cali files, you may create a folder with a different name and replace the folder name here
tkSorted = th.Thicket.from_caliperreader("./results/cali");
tkRandom = th.Thicket.from_caliperreader("resultsFinal/caliRandom");
tkReverse = th.Thicket.from_caliperreader("resultsFinal/caliReverse");
tkPerturbed = th.Thicket.from_caliperreader("resultsFinal/caliPerturbed");

In [ ]:
print(tkSorted.tree(metric_column="Avg time/rank", precision=5))

In [ ]:
tkRandom.metadata_column_to_perfdata("num_procs")
tkRandom.metadata_column_to_perfdata("input_size")

tkRandom.dataframe = tkRandom.dataframe.reset_index().set_index(["node", "num_procs", "input_size"]).sort_index()

tkSorted.metadata_column_to_perfdata("num_procs")
tkSorted.metadata_column_to_perfdata("input_size")

tkSorted.dataframe = tkSorted.dataframe.reset_index().set_index(["node", "num_procs", "input_size"]).sort_index()

tkReverse.metadata_column_to_perfdata("num_procs")
tkReverse.metadata_column_to_perfdata("input_size")

tkReverse.dataframe = tkReverse.dataframe.reset_index().set_index(["node", "num_procs", "input_size"]).sort_index()

tkPerturbed.metadata_column_to_perfdata("num_procs")
tkPerturbed.metadata_column_to_perfdata("input_size")

tkPerturbed.dataframe = tkPerturbed.dataframe.reset_index().set_index(["node", "num_procs", "input_size"]).sort_index()

In [ ]:
tkRandom.dataframe

In [ ]:
tkSorted.dataframe

In [ ]:
tkReverse.dataframe

In [ ]:
tkPerturbed.dataframe

In [ ]:
tkSorted.metadata

In [ ]:
tkReverse.metadata

In [ ]:
tkPerturbed.metadata

In [ ]:
leg = ["MIN", "AVG", "MAX"]

processes = [2, 4, 8, 16, 32, 64, 128, 256, 512, 1024]

def plot_mplRandom(df, df2, df3, title, xlabel, ylabel):
    print(df.head())  
    
    for input_size in df.columns:
        fig = plt.figure(figsize=(15,7))
        ax = fig.add_subplot(1, 1, 1)
        ax.plot(processes, df[input_size], marker='o')
        ax.plot(processes, df2[input_size], marker='o')
        ax.plot(processes, df3[input_size], marker='o')

        ax.set_title(title + ", Random, Input size: " + str(input_size))
        ax.set_xlabel(xlabel)
        ax.set_ylabel(ylabel)
        ax.legend(leg)
        plt.xscale("log", base=2)
        plt.xticks(processes)
        ax.set_xticklabels(processes)
        plt.show()
        
        
for node in tkRandom.graph.traverse():
    plot_mplRandom(
        df=tkRandom.dataframe.loc[node, "Min time/rank"].unstack(level="input_size"),
        df2=tkRandom.dataframe.loc[node, "Avg time/rank"].unstack(level="input_size"),
        df3=tkRandom.dataframe.loc[node, "Max time/rank"].unstack(level="input_size"),
        title=node.frame["name"],
        xlabel="Processes",
        ylabel="Time (seconds)",
    )

def plot_mplRandomOne(df, title, xlabel, ylabel, typ):
    print(df.head())  
    
    for input_size in df.columns:
        fig = plt.figure(figsize=(15,7))
        ax = fig.add_subplot(1, 1, 1)
        ax.plot(processes, df[input_size], marker='o')
        ax.set_title(title + " " + typ + ", Random, Input size: " + str(input_size))
        ax.set_xlabel(xlabel)
        ax.set_ylabel(ylabel)
        #ax.legend(leg)
        plt.xscale("log", base=2)
        plt.xticks(processes)
        ax.set_xticklabels(processes)
        plt.show()
    
    
for node in tkRandom.graph.traverse():
    plot_mplRandomOne(
        df=tkRandom.dataframe.loc[node, "Total time"].unstack(level="input_size"),
        title=node.frame["name"],
        xlabel="Processes",
        ylabel="Time (seconds)",
        typ="Total Time"
    )
    
for node in tkRandom.graph.traverse():
    plot_mplRandomOne(
        df=tkRandom.dataframe.loc[node, "Variance time/rank"].unstack(level="input_size"),
        title=node.frame["name"],
        xlabel="Processes",
        ylabel="Time (seconds)",
        typ="Variance"
    )
    

In [ ]:
leg = ["MIN", "AVG", "MAX"]

processes = [2, 4, 8, 16, 32, 64, 128, 256, 512, 1024]

def plot_mplSorted(df, df2, df3, title, xlabel, ylabel):
    print(df.head())  
    
    for input_size in df.columns:
        fig = plt.figure(figsize=(15,7))
        ax = fig.add_subplot(1, 1, 1)
        ax.plot(processes, df[input_size], marker='o')
        ax.plot(processes, df2[input_size], marker='o')
        ax.plot(processes, df3[input_size], marker='o')

        ax.set_title(title + ", Sorted, Input size: " + str(input_size))
        ax.set_xlabel(xlabel)
        ax.set_ylabel(ylabel)
        ax.legend(leg)
        plt.xscale("log", base=2)
        plt.xticks(processes)
        ax.set_xticklabels(processes)
        plt.show()

for node in tkSorted.graph.traverse():
    plot_mplSorted(
        df=tkSorted.dataframe.loc[node, "Min time/rank"].unstack(level="input_size"),
        df2=tkSorted.dataframe.loc[node, "Avg time/rank"].unstack(level="input_size"),
        df3=tkSorted.dataframe.loc[node, "Max time/rank"].unstack(level="input_size"),
        title=node.frame["name"],
        xlabel="Processes",
        ylabel="Time (seconds)",
    )
    
def plot_mplSortedOne(df, title, xlabel, ylabel, typ):
    print(df.head())  
    
    for input_size in df.columns:
        fig = plt.figure(figsize=(15,7))
        ax = fig.add_subplot(1, 1, 1)
        ax.plot(processes, df[input_size], marker='o')
        ax.set_title(title + " " + typ + ", Sorted, Input size: " + str(input_size))
        ax.set_xlabel(xlabel)
        ax.set_ylabel(ylabel)
        #ax.legend(leg)
        plt.xscale("log", base=2)
        plt.xticks(processes)
        ax.set_xticklabels(processes)
        plt.show()
    
    
for node in tkSorted.graph.traverse():
    plot_mplSortedOne(
        df=tkSorted.dataframe.loc[node, "Total time"].unstack(level="input_size"),
        title=node.frame["name"],
        xlabel="Processes",
        ylabel="Time (seconds)",
        typ="Total Time"
    )
    
for node in tkSorted.graph.traverse():
    plot_mplSortedOne(
        df=tkSorted.dataframe.loc[node, "Variance time/rank"].unstack(level="input_size"),
        title=node.frame["name"],
        xlabel="Processes",
        ylabel="Time (seconds)",
        typ="Variance"
    )

In [ ]:
leg = ["MIN", "AVG", "MAX"]

processes = [2, 4, 8, 16, 32, 64, 128, 256, 512, 1024]

def plot_mplReverse(df, df2, df3, title, xlabel, ylabel):
    print(df.head())  
    
    for input_size in df.columns:
        fig = plt.figure(figsize=(15,7))
        ax = fig.add_subplot(1, 1, 1)
        ax.plot(processes, df[input_size], marker='o')
        ax.plot(processes, df2[input_size], marker='o')
        ax.plot(processes, df3[input_size], marker='o')

        ax.set_title(title + ", Reverse, Input size: " + str(input_size))
        ax.set_xlabel(xlabel)
        ax.set_ylabel(ylabel)
        ax.legend(leg)
        plt.xscale("log", base=2)
        plt.xticks(processes)
        ax.set_xticklabels(processes)
        plt.show()

for node in tkReverse.graph.traverse():
    plot_mplReverse(
        df=tkReverse.dataframe.loc[node, "Min time/rank"].unstack(level="input_size"),
        df2=tkReverse.dataframe.loc[node, "Avg time/rank"].unstack(level="input_size"),
        df3=tkReverse.dataframe.loc[node, "Max time/rank"].unstack(level="input_size"),
        title=node.frame["name"],
        xlabel="Processes",
        ylabel="Time (seconds)",
    )
    
def plot_mplReverseOne(df, title, xlabel, ylabel, typ):
    print(df.head())  
    
    for input_size in df.columns:
        fig = plt.figure(figsize=(15,7))
        ax = fig.add_subplot(1, 1, 1)
        ax.plot(processes, df[input_size], marker='o')
        ax.set_title(title + " " + typ + ", Reverse, Input size: " + str(input_size))
        ax.set_xlabel(xlabel)
        ax.set_ylabel(ylabel)
        #ax.legend(leg)
        plt.xscale("log", base=2)
        plt.xticks(processes)
        ax.set_xticklabels(processes)
        plt.show()
    
    
for node in tkReverse.graph.traverse():
    plot_mplReverseOne(
        df=tkReverse.dataframe.loc[node, "Total time"].unstack(level="input_size"),
        title=node.frame["name"],
        xlabel="Processes",
        ylabel="Time (seconds)",
        typ="Total Time"
    )
    
for node in tkReverse.graph.traverse():
    plot_mplReverseOne(
        df=tkReverse.dataframe.loc[node, "Variance time/rank"].unstack(level="input_size"),
        title=node.frame["name"],
        xlabel="Processes",
        ylabel="Time (seconds)",
        typ="Variance"
    )

In [ ]:
leg = ["MIN", "AVG", "MAX"]

processes = [2, 4, 8, 16, 32, 64, 128, 256, 512, 1024]

def plot_mplPerturbed(df, df2, df3, title, xlabel, ylabel):
    print(df.head())  
    
    for input_size in df.columns:
        fig = plt.figure(figsize=(15,7))
        ax = fig.add_subplot(1, 1, 1)
        ax.plot(processes, df[input_size], marker='o')
        ax.plot(processes, df2[input_size], marker='o')
        ax.plot(processes, df3[input_size], marker='o')

        ax.set_title(title + ", Perturbed, Input size: " + str(input_size))
        ax.set_xlabel(xlabel)
        ax.set_ylabel(ylabel)
        ax.legend(leg)
        plt.xscale("log", base=2)
        plt.xticks(processes)
        ax.set_xticklabels(processes)
        plt.show()

for node in tkPerturbed.graph.traverse():
    plot_mplPerturbed(
        df=tkPerturbed.dataframe.loc[node, "Min time/rank"].unstack(level="input_size"),
        df2=tkPerturbed.dataframe.loc[node, "Avg time/rank"].unstack(level="input_size"),
        df3=tkPerturbed.dataframe.loc[node, "Max time/rank"].unstack(level="input_size"),
        title=node.frame["name"],
        xlabel="Processes",
        ylabel="Time (seconds)",
    )
    
def plot_mplPerturbedOne(df, title, xlabel, ylabel, typ):
    print(df.head())  
    
    for input_size in df.columns:
        fig = plt.figure(figsize=(15,7))
        ax = fig.add_subplot(1, 1, 1)
        ax.plot(processes, df[input_size], marker='o')
        ax.set_title(title + " " + typ + ", Perturbed, Input size: " + str(input_size))
        ax.set_xlabel(xlabel)
        ax.set_ylabel(ylabel)
        #ax.legend(leg)
        plt.xscale("log", base=2)
        plt.xticks(processes)
        ax.set_xticklabels(processes)
        plt.show()
    
    
for node in tkPerturbed.graph.traverse():
    plot_mplPerturbedOne(
        df=tkPerturbed.dataframe.loc[node, "Total time"].unstack(level="input_size"),
        title=node.frame["name"],
        xlabel="Processes",
        ylabel="Time (seconds)",
        typ="Total Time"
    )
    
for node in tkPerturbed.graph.traverse():
    plot_mplPerturbedOne(
        df=tkPerturbed.dataframe.loc[node, "Variance time/rank"].unstack(level="input_size"),
        title=node.frame["name"],
        xlabel="Processes",
        ylabel="Time (seconds)",
        typ="Variance"
    )